<a href="https://colab.research.google.com/github/zfukuoka/Training-KikaiGakusyuZukan/blob/master/Chap2_02_RidgeRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第２章  教師あり学習

## 02 正則化

ポイント

* 正則化は過学習を防ぐための手法の一つで線形回帰のアルゴリズムと共に利用
* テキストでは y = sin(2 * pi * x) に乱数を加えて生成したデータを元に学習に用いたデータと検証データを分けたうえで、学習誤差と検証誤差をまとめた例で説明
  * 線形回帰の次元を増やしていくと学習誤差は小さくなる一方、検証誤差は大きくなる結果

| 次数 | 学習誤差 | 検証誤差 |
| :--- | :---: | :---: |
| 1 | 0.412 | 0.618 |
| 2 | 0.176 | 0.193 |
| 3 | 0.081 | 0.492 |
|   |       |       |
| 6 | 0.024 | 3.472 | 

* 上記の事例に正則化したら、下記のようになったという結果

| 次数 | 学習誤差 | 検証誤差 |
| :--- | :---: | :---: |
| 1 | 0.412 | 0.612 |
| 2 | 0.372 | 0.532 |
| 3 | 0.301 | 0.394 |
|   |       |       |
| 6 | 0.159 | 0.331 |

* 複雑なモデル（次数が高いモデル）が過学習する様子と過学習を防げる様子は上記の事例で確認
  * 過学習の原因として、学習パラメータ w1 が極端に大きい（または小さい）値を取ることがある

| 次数 | w0 | w1 | w2 | w3 | w4 | w5 | w6 |
| :--- | ---: | ---: | ---: | ---: | ---: | ---: | ---: |
| 1 | -0.007 | -0.217 |        |        | | | |
| 2 |  0.978 | -5.222 |  4.204 |        | | | |
| 3 |  0.281 |  4.927 |-17.639 | 12.157 | | | |
| 6 |  1.080 |-26.324 |287.431 |-1024.141|1611.144|-1147.946|308.643|

* 上記の事例に正則化したら、下記のようになったという結果

| 次数 | w0 | w1 | w2 | w3 | w4 | w5 | w6 |
| :--- | ---: | ---: | ---: | ---: | ---: | ---: | ---: |
| 1 | -0.055 | -0.149 |        |        | | | |
| 2 | -0.066 | -0.493 |  0.421 |        | | | |
| 3 | -0.001 | -0.716 | -0.042 |  0.670 | | | |
| 6 |  0.191 | -0.751 | -0.497 | -0.182|  0.109 |  0.370 |  0.607|

* 正則化で学習パラメータを抑えることができるのかRidge回帰の誤差関数（2次関数の場合）
  * 右辺の第1項は線形回帰の損失関数
  * 右辺の第2講は罰則項（または正規化項）と呼ばれるもので、学習パラメータの2乗和
    * 切片であるw0 は罰則項に含まないのが一般的
    * α( >= 0) は正規化の強さをコントロールするパラメータでαが大きいほど学習パラメータが抑えられ、αが小さいほど学習データへの当てはまりが強くなる
  * Ridge回帰の損失関数R(w)を最小化するには...
    * 右辺の第1項だけに注目すると、学習データy との誤差を小さくするような任意の w0, w1, w2 を問題になる
    * しかし、右辺の第2項の罰則項は学習パラメータの2乗和になっているので、学習パラメータの絶対値が大きいと、損失関数全体が大きくなってしまう
      * 結局のところ罰則項は学習パラメータの絶対値が大きくなることに対して損失が大きくなるような罰則を与える役割を持っており、この役割で学習パラメータを抑えている

$$
R(w) = 
\sum_{i=1}^{n} 
  \Bigl( 
    y_i
    \bigl(
      w_0 + w_1 x_i + w_2 x_i^2
    \bigr)
  \Bigr)^2 + \alpha \bigl(
    w_1^2 + w_2^2
  \bigr)
$$


* サンプルコードのポイントはあとで書く

In [0]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
